<a href="https://colab.research.google.com/github/DrSantanche/MachineLearningGBM/blob/master/ML_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Forecasting Brownian Motion using SVM and LSTM Neural Network**

In this script, I will define two machine learning models to forecast a randomly generated geometric brownian motion.
The forecast will be directional only, so the problem will be defined as classification.

The performance will be evaluated through accuracy, since, of course, the cost of false negatives and false positives is equal.

# First model: SVM

In this section, I will initially load the required packages and then define the data generation process and the SVM classifier as custom classes.

The data will be split into training, validation and test set, where the validation happens to optimize the kernel function used, since the SVMs are usually sensitive to it.

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics


class GeomBrownianMotion:

  #Initialization.
  #Parameters of GBM:
  #S0,mu,sigma,n_sim,T

  #We will assume dt = 1 (day) 
  def __init__(self,S0,mu,sigma,n_sim,T,seed=None):
    if seed != None:
      np.random.seed(seed)
    self.dWt = np.random.standard_normal(n_sim-1)
    t = np.linspace(0,T,n_sim)
    self.dt = float(T)/(n_sim-1)
    self.Wt = np.cumsum(self.dWt)*np.sqrt(self.dt)
    self.Wt = np.insert(self.Wt,0,0)
      
    #Use Wt to create the brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*self.Wt
    self.S = S0*np.exp(X) ### geometric brownian motion ###
    self.dS = (np.log(self.S[1:])-np.log(self.S[:-1]))*100

This first class implements the GBM. Of course, this type of input is impossible to forecast by definition, so the performance of any model should be close to the 50% area. The main target, anyway, is the **dS** attribute, since we would have a stationary time series of log returns, normally easier to predict.

Here, instead, the definition of the data class:

In [0]:

class MyData:
    
    def __init__(self,ts,step):
        
        self.timeseries = ts
        
        self.step = step
        
    def prepareInputOutput(self,train_size,val_size,test_size):
        
        self.X = []; self.y = []
        
        for i in range(self.step,len(self.timeseries)-1):
            
            temp = np.array(self.timeseries[(i-self.step):i])
            
            self.X.append(temp)
            
            self.y.append(np.sign(self.timeseries[i+1]))
            
        val_prop = val_size/(train_size + val_size)
    
        X_train, self.X_test, y_train, self.y_test = train_test_split(self.X, self.y, test_size = test_size)
        
        if val_prop > 0:
        
            self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(X_train, y_train, 
                                                          test_size = val_prop)


In this class, we initialize an object by adding our time series and the step we want to implement in order to define input and output for the ML models.

For example, having a time step of 5 would mean to have vectors of 5 elements as input, and an output of 1 elemen (the next return to forecast). In the *prepareInputOutput* function, it is defined how to create our training, validation and test set, splitting them after collecting the vectors in the proper way based on time step. Please note that we allow the validation set to be empty.

The definition of the SVM class is as below:


In [0]:
class SVM_forecast:
    
    def __init__(self,data):
        
        self.data = data
        
    def chooseKernel(self):
        
        #I would like to run the task for the three most common kernel functions,
        #the linear, polynomial (using 3rd order) and rbf
        kernels = ["linear","poly","rbf"]
        
        self.accuracyVal = -1 #Initialized to update
        
        for k in kernels:
            
            classif = svm.SVC(kernel = k,  gamma = "auto")
            
            classif.fit(self.data.X_train, self.data.y_train)
            
            y_for = classif.predict(self.data.X_val)
            
            Acc = metrics.accuracy_score(self.data.y_val,y_for)
            
            if Acc > self.accuracyVal:
                
                self.optKernel = k
                
                self.accuracyVal = Acc
                
    def testPerformance(self):
        
        self.classifier = svm.SVC(kernel = self.optKernel, gamma = "auto")
        
        self.classifier.fit(self.data.X_train, self.data.y_train)
        
        self.forecasts = self.classifier.predict(self.data.X_test)
        
        self.accuracyTest = metrics.accuracy_score(self.data.y_test, self.forecasts)


We have two main functions here, the validator function (testing the kernel functions and choosing the best based on accuracy) and the testing function which tells us the performance after having chosen a model.

Here is the implementation of the model:

In [0]:
Years = 3

L = Years * 252
        
GBM = GeomBrownianMotion(100,0.02,0.03,L,1)

print("GBM complete")

Data = MyData(GBM.dS,5)

Data.prepareInputOutput(0.6,0.2,0.2)

print("Data complete")

Model = SVM_forecast(Data)

Model.chooseKernel()

print("Model choice complete")

Model.testPerformance()

print("Optimal kernel: " + Model.optKernel)

print("Accuracy on testset: " + str(Model.accuracyTest))

GBM complete
Data complete
Model choice complete
Optimal kernel: poly
Accuracy on testset: 0.4866666666666667


Defined a number of years, we obtain the number of business days. Those will be our data points.

The example shows how to implement a time step of 5 on a dataset divided into 60% training set, 20% validation set and 20% test set. As expected, the performance is about 50% with small deviations.

# Deep Learning: LSTM Network

In this section, I use the previously generated Geometric Brownian Motion and Data classes in a classification exercise using a LSTM Network.

In this first part, we load the libraries and define our LSTM class:

In [0]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

class LSTMNetwork:
    
    def __init__(self,num_layers,data,dropOut=True):
        
        self.model = Sequential()
        
        self.data = data
        
        self.Scaler = StandardScaler()
        
        #Converting data to right format
        self.data.X = np.vstack(self.data.X)
        
        self.data.y = np.vstack(self.data.y)
        
        self.data.X_train = np.vstack(self.data.X_train)
        
        self.data.y_train = np.vstack(self.data.y_train)
        
        self.data.X_test = np.vstack(self.data.X_test)
        
        self.data.y_test = np.vstack(self.data.y_test)
        
        #Obtaining useful objects to reshape for network
        self.features = self.data.X.shape[1]
        
        self.dimtrain = self.data.X_train.shape[0]
        
        self.dimtest = self.data.X_test.shape[0]
        
        for i in range(num_layers):

            self.model.add(LSTM(units=50, return_sequences=True, input_shape=(1, self.features)))
            
            if dropOut:
                
                self.model.add(Dropout(0.2))
            
        self.model.add(Dense(units = 1, activation = "softmax"))
        
        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ["accuracy"])
            
    def training(self):
        
        #Scaled data
        
        self.data.X_train = self.Scaler.fit_transform(self.data.X_train).reshape(self.dimtrain,1,self.features)
                
        self.data.y_train = self.data.y_train.reshape(self.dimtrain,1,1)
        
        self.model.fit(self.data.X_train, 
                       self.data.y_train, epochs = 100, batch_size=32)
        
    def testPerformance(self):
        
        self.data.X_test = self.Scaler.transform(self.data.X_test).reshape(self.dimtest,1,self.features)
                
        self.data.y_test = self.data.y_test.reshape(self.dimtest,1,1)
        
        self.accuracyTest = self.model.evaluate(self.data.X_test,self.data.y_test,batch_size = 32)[1]

        

The format of the data is slightly different, as Keras allows for several samples on the same time and feature dimension. In particular, both input and output should be (n_samples, n_time, n_features) dimensional. Of course, the features of the output are in this case 1 (the next data point).

Additionally, here I will skip the validation, however it is still possible to implement it similarly to what we did before. The validation may happen for example on the number of layers.

Here the implementation of the model:

In [0]:
#Deep learning LSTM Network

Data.prepareInputOutput(0.8,0.,0.2)

Model_NN = LSTMNetwork(4,Data)

print("Network initialized")

Model_NN.training()

print("Network trained")

Model_NN.testPerformance()

print("Accuracy on test set is: " + str(Model_NN.accuracyTest))

Network initialized
Epoch 1/100
449/449 [==============================] - 6s 13ms/step - loss: 16.1199 - acc: 0.4944
Epoch 2/100
449/449 [==============================] - 0s 367us/step - loss: 16.1199 - acc: 0.4944
Epoch 3/100
449/449 [==============================] - 0s 367us/step - loss: 16.1199 - acc: 0.4944
Epoch 4/100
449/449 [==============================] - 0s 355us/step - loss: 16.1199 - acc: 0.4944
Epoch 5/100
449/449 [==============================] - 0s 354us/step - loss: 16.1199 - acc: 0.4944
Epoch 6/100
449/449 [==============================] - 0s 364us/step - loss: 16.1199 - acc: 0.4944
Epoch 7/100
449/449 [==============================] - 0s 362us/step - loss: 16.1199 - acc: 0.4944
Epoch 8/100
449/449 [==============================] - 0s 350us/step - loss: 16.1199 - acc: 0.4944
Epoch 9/100
449/449 [==============================] - 0s 433us/step - loss: 16.1199 - acc: 0.4944
Epoch 10/100
449/449 [==============================] - 0s 364us/step - loss: 16.1199 - ac

As we can see, the accuracy is still very close to 50%.

# Conclusions

In this exercise we have implemented SVM and LSTM frameworks to forecast the future returns of our time series (thus, if the price will go up or down, as it is only based on the sign of the return).

The performance in this case doesn't really tell us much, since the GBM is unpredictable, but the way we implemented these models can actually be applied on more meaningful time series.